In [21]:
%%writefile QuizApp.java
import java.util.*;
import java.util.Timer;
import java.util.TimerTask;
import java.util.concurrent.atomic.AtomicBoolean;
import java.util.concurrent.atomic.AtomicReference;
import java.io.*;

public class QuizApp {
    private static int score = 0;
    private static int totalQuestions = 5;
    private static Scanner scanner = new Scanner(System.in);
    private static Timer timer;

    private static Question[] questions = {
        new Question("What is the lifespan of a mosquito ?", new String[]{"A) 10 to 100 days", "B) 10 to 56 days", "C) 5 to 50 days", "D) 1 to 60 days"},'B'),
        new Question("Which language is used for web development?", new String[]{"A) Python", "B) HTML", "C) Java", "D) C++"},'B'),
        new Question(" Who is called the king of fruits ?", new String[]{"A) Mango", "B) TO apple", "C) papaya", "D) Coconut"},'A'),
        new Question("Largest planet in solar system?", new String[]{"A) Earth", "B) Mars", "C) Jupiter", "D) Saturn"},'C'),
        new Question("In Which county was the world's first clock made ?", new String[]{"A) America", "B) Japan", "C) England", "D) India"},'C')
    };

    static class Question {
        String question;
        String[] options;
        char correctAnswer;

        Question(String q, String[] opts, char ans) {
            this.question = q;
            this.options = opts;
            this.correctAnswer = ans;
        }
    }

    public static void main(String[] args) {
        System.out.println("=== Welcome to Java Quiz App ===");
        System.out.println("Each question has 15 seconds. Good luck!\n");

        for (int i = 0; i < totalQuestions; i++) {
            askQuestion(questions[i], i + 1);
        }

        showResults();
        scanner.close();
    }

    private static void askQuestion(Question q, int questionNum) {
        final AtomicBoolean timeUp = new AtomicBoolean(false);
        final AtomicReference<String> userInput = new AtomicReference<>("");
        final Thread inputThread = new Thread(() -> {
            try {
                // Synchronize access to scanner to prevent concurrent modification
                synchronized (scanner) {
                    if (scanner.hasNextLine()) {
                        userInput.set(scanner.nextLine().trim().toUpperCase());
                    }
                }
            } catch (NoSuchElementException | IllegalStateException e) {
                // Ignore, happens if scanner is closed or input stream ends
            } catch (Exception e) {
                // General exception handler, including InterruptedException if thread is interrupted
            }
        });

        // Clear screen (Linux/Mac)
        System.out.print("\u001B[H\u001B[2J");

        // Display question
        System.out.println("\n--- Question " + questionNum + " ---");
        System.out.println(q.question);
        for (String option : q.options) {
            System.out.println(option);
        }
        System.out.println("Your answer (A/B/C/D): ");

        // Start timer in separate thread
        timer = new Timer();
        timer.scheduleAtFixedRate(new TimerTask() {
            int secondsLeft = 15;
            @Override
            public void run() {
                if (secondsLeft > 0) {
                    System.out.print("\nTime left: " + secondsLeft + "s     ");
                    secondsLeft--;
                } else {
                    timeUp.set(true);
                    System.out.println("\n\nTIME'S UP! Moving to next question...");
                    // If time is up, interrupt the input thread to stop its blocking nextLine() call.
                    if (inputThread.isAlive()) {
                        inputThread.interrupt();
                    }
                    timer.cancel();
                }
            }
        }, 1000, 1000);

        inputThread.start();

        // Wait for input thread to finish or for a maximum of 15 seconds
        try {
            inputThread.join(15500); // Wait for input thread, or timeout
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }

        timer.cancel(); // Ensure timer is cancelled if input thread finishes early

        char userAnswer = userInput.get().isEmpty() ? ' ' : userInput.get().charAt(0);
        checkAnswer(q, userAnswer, timeUp.get());
    }

    private static void checkAnswer(Question q, char userAnswer, boolean timeUp) {
        System.out.println("\nYou answered: '" + userAnswer + "' | Correct: '" + q.correctAnswer + "'");

        if (timeUp || userAnswer != q.correctAnswer) {
            if (timeUp) {
                System.out.println("❌ WRONG - Time expired!");
            } else {
                System.out.println("❌ WRONG!");
            }
        } else {
            score++;
            System.out.println("✅ CORRECT! +1 point");
        }
        System.out.println("Press Enter to continue...");
        // Removed scanner.nextLine() here to prevent IndexOutOfBoundsException
        // The prompt will still display, but won't block for user input.
        // If explicit blocking is desired, a more robust input handling mechanism would be needed.
    }

    private static void showResults() {
        double percentage = (double) score / totalQuestions * 100;
        System.out.println("\n=== QUIZ COMPLETED ===");
        System.out.println("Final Score: " + score + "/" + totalQuestions);
        System.out.printf("Percentage: %.1f%%\n", percentage);

        if (percentage >= 80) System.out.println("🎉 Excellent!");
        else if (percentage >= 60) System.out.println("👍 Good job!");
        else System.out.println("📚 Keep practicing!");
    }
}

Overwriting QuizApp.java


In [22]:
!javac QuizApp.java
!java QuizApp






=== Welcome to Java Quiz App ===
Each question has 15 seconds. Good luck!


--- Question 1 ---
What is the lifespan of a mosquito ?
A) 10 to 100 days
B) 10 to 56 days
C) 5 to 50 days
D) 1 to 60 days
Your answer (A/B/C/D): 

Time left: 15s     
Time left: 14s     
Time left: 13s     
Time left: 12s     
Time left: 11s     
Time left: 10s     
Time left: 9s     B

You answered: 'B' | Correct: 'B'
✅ CORRECT! +1 point
Press Enter to continue...

--- Question 2 ---
Which language is used for web development?
A) Python
B) HTML
C) Java
D) C++
Your answer (A/B/C/D): 

Time left: 15s     
Time left: 14s     
Time left: 13s     
Time left: 12s     B

You answered: 'B' | Correct: 'B'
✅ CORRECT! +1 point
Press Enter to continue...

--- Question 3 ---
 Who is called the king of fruits ?
A) Mango
B) TO apple
C) papaya
D) Coconut
Your answer (A/B/C/D): 

Time left: 15s     
Time left: 14s     
Time left: 13s     
Time left: 12s     
Time left: 11s     
Time left: 10s     
Time left: 9s     
Time left